***

# VERSION PROJET SCRAPY EXTERNE
## Guide pas-à-pas : Créer et lancer un projet Scrapy pour IMDb

## 1. Création automatique de l’arborescence du projet

Se placer avec le terminal (bash / cmd / PowerShell) la où on veut que Scrapy crée le projet :
ex:
```bash
cd semaine2\scrapy\movie_director_exercice
```
Créer le projet Scrapy avec la commande suivante (remplace "imdb_scraper" par le nom de ton projet) :
```bash
scrapy startproject imdb_scraper
```
La sortie doit etre similaire à ceci :
```
New Scrapy project 'imdb_scraper', using template directory 'C:\Users\pkoub\anaconda3\envs\jedha-scrapy\Lib\site-packages\scrapy\templates\project', created in:
    C:\Projects\formation\jedha\dsfs-ft-40\semaine2\scrapy\imdb_scraper

You can start your first spider with:
    cd imdb_scraper
    scrapy genspider example example.com
```
L'arborescence suivante a été générée par Scrapy :
```bash
imdb_scraper/                     ← Dossier racine du projet (celui à ouvrir dans VS Code)
├── scrapy.cfg                    ← Fichier de config global (Scrapy le lit depuis ici)
└── imdb_scraper/                 ← Module Python importable (le "vrai" package)
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py               ← Les réglages du projet
    └── spiders/
        ├── __init__.py
        └── spiders (tes spiders ici)
```

#### Pourquoi Scrapy crée-t-il 2 dossiers "imdb_scraper" l’un dans l’autre ?
-   Le dossier extérieur (imdb_scraper/ racine)
    -   C’est juste un conteneur pour tout le projet: il contient scrapy.cfg (le fichier qui dit à Scrapy "je suis un projet Scrapy")
    -   Tu lances les commandes (scrapy crawl, scrapy genspider, etc.) depuis ce dossier
    -   C’est le dossier que tu versionnes avec Git, que tu mets sur **GitHub**, que tu déploies, etc.
-   Le dossier intérieur (imdb_scraper/imdb_scraper/)
    -   C’est un vrai **module** Python (package)
    -   Il a un __init__.py → Python le reconnaît comme un package importable
    -   ***C’est depuis ce dossier que tu fais les imports dans tes spiders (crée le dossier 'utils' dans le 2e 'imdb_scraper/') :***
``` 
        from imdb_scraper.items import MonItem
        from imdb_scraper.utils.parsing import parse_vote_count
```
Se placer dans le dossier racine du projet (le 1er imdb_scraper/) pour lancer les commandes Scrapy :
```bash
imdb_scraper/                     ← Dossier racine du projet (celui à ouvrir dans VS Code)
├── scrapy.cfg                    
└── imdb_scraper/                 
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py               
    └── spiders/
        ├── __init__.py
        └── spiders (tes spiders ici)
```

## 2. Création du spider imdb1 (ou le nom que tu veux)

Ouvrir un terminal (bash / cmd / PowerShell) et taper :

```bash
scrapy scrapy genspider imdb1 imdb.com
```
#### ceci génère un fichier 'imdb1.py'(le spider) dans le dossier **spiders**.
-   Modifier / compléter ce fichier qui contient une classe **Imdb1Spider** qui hérite de **scrapy.Spider**
-   Vérifier avec :
```bash
scrapy list
```
La sortie doit afficher le nom de ton spider :
```
imdb1
```
nb: si vous modifiez le nom de la classe du spider, vérifiez que le nom du spider (attribut name) correspond au nom de la classe en minuscules sans "Spider" (ex: class Imdb1Spider → name = "imdb1")

## 3. Modifier les fichiers **settings.py** et **imdb1.py**

    -   `USER_AGENT` = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    -   `FEED`
    -   ...
    --> On ne les met plus dans une classe !
## 4. Lancement du crawling et collecte des données

Ouvrir un terminal (bash / cmd / PowerShell) et taper :

```bash
scrapy crawl imdb1 -o output/imdb.json -L INFO
```

-   -o output/imdb.json : indique à Scrapy de sauvegarder les résultats dans un fichier JSON (dans le dossier output/)
-   -L INFO : définit le niveau de logging à INFO pour avoir des logs plus lisibles pendant le crawling"

## 5. check du résultat de la collecte -> le fichier de sortie
-   Ouvrir le fichier `output/imdb.json` pour vérifier que les données ont bien été collectées et sauvegardées au format JSON.

## exemple de classe pour un projet scrapy externe (imdb_scraper/imdb_scraper/spiders/imdb1.py) :
Pour info: elle sera lancée pour recupérer des données depuis le ***dossier racine du projet*** (le 1er imdb_scraper/) avec la commande 
```bash
scrapy crawl imdb1 -o output/imdb.json -L INFO`
```
#### la version pour utilisation dans notebook est laissée en commentaire pour comparer les deux versions. les import sont gérés par Scrapy, ainsi que les logs, et beaucoup d'autres fonctionnalités (gestion des erreurs, des redirections, des requetes multiples, etc. --> infos dans le fichier settings.py)


In [ ]:
import scrapy
# import requests              # <-- supprimé : on n'utilise plus requests dans un spider Scrapy
# import logging
# import os
# from scrapy.crawler import CrawlerProcess          # <-- pas besoin ici (lancé via commande scrapy crawl)
# from parsel import Selector                        # <-- pas besoin (response est déjà un Selector)

# On suppose que parse_vote_count est dans utils/parsing.py
#pour externaliser le code de parsing et le rendre réutilisable dans d'autres spiders du projet
# créer un repertoire (ex utils/) et un fichier "parsing.py" pour y mettre le code "def votre_fonction(): ..."
#      l'importer avec 
#                   from imdb_scraper.utils.parsing import votre_fonction

# Version locale de parse_vote_count (recommandée pour commencer)
def parse_vote_count(vote_parts):
    """
    Extrait le nombre de votes à partir des parties extraites.
    Exemple : [' (', '3,2\xa0M', ')'] → 3200000
    """
    if len(vote_parts) < 3:
        return 0
    
    raw = vote_parts[1].replace('\xa0', '').strip()  # '3,2M'
    
    multi = 1
    if 'M' in raw.upper():
        multi = 1_000_000
        raw = raw.rstrip('Mm')
    elif 'K' in raw.upper():
        multi = 1_000
        raw = raw.rstrip('Kk')
    
    raw = raw.replace(',', '.').strip()
    
    try:
        return int(float(raw) * multi)
    except (ValueError, TypeError):
        return 0


class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    
    start_urls = ["https://www.imdb.com/chart/top/"]
    
    def parse(self, response):
        classement = 0     
        
        for film in response.css("li.ipc-metadata-list-summary-item"):
            classement += 1
            
            # ### On récupère le titre
            titre_raw = film.css("h3.ipc-title__text::text").get()
            titre = titre_raw.split('. ', 1)[-1].strip() if titre_raw else '?'  # ← nettoyé
            
            # ### On récupère l'URL
            # base_url="https://www.imdb.com"
            film_relative_url = film.css('a.ipc-title-link-wrapper::attr(href)').get('')
            if not film_relative_url:
                continue
            
            # url_details = response.urljoin(film_relative_url)   # ← on le fait dans Request
            
            # ### On récupère le rating
            # rating = film.css("span.ipc-rating-star--rating::text").get(), mais on prefere le data-testid plus robuste
            # pour info le.get(default='?') permet de retourner '?' si le sélecteur ne trouve rien, au lieu de retourner 'None'
            rating = film.css('span[data-testid="ratingGroup--imdb-rating"] ::text').get(default='?').strip()
            
            # ### On récupère le nombre de votants
            vote_parts = film.css("span.ipc-rating-star--voteCount::text").getall()
            nb_voteurs = parse_vote_count(vote_parts)
            
            # ### On a tout scrappé sur la 1ere page, on avance vers le lien detail du film
            # pour scraper le revenu brut mondial
            # detail_resp = requests.get(url_details, headers=headers, timeout=10)
            # detail_sel = Selector(text=detail_resp.text)    
            # revenu_brut_mondial = detail_sel.css( ... ).get(default='?').strip()
            # → Remplacé par :
           
            yield scrapy.Request(
                url=response.urljoin(film_relative_url),
                callback=self.parse_detail,
                meta={
                    'classement': classement,
                    'titre': titre,
                    'rating': rating,
                    'nb_voteurs': nb_voteurs,
                }
            )
            
            """ ce YIELD ci-dessus signifie : Quand tu auras téléchargé et parsé cette nouvelle page (la page détail du film),
            appelle automatiquement la méthode parse_detail et passe-lui la réponse. 
            
            Pour suivre les liens je pourrais aussi faire un 'response.follow' :
            yield response.follow(
                url=film_relative_url,              # relatif ou absolu
                callback=self.parse_detail,
                meta={...}
            )
            """


    # ici on estparti de la 1ere page et on est dans
    # la page détail du film, on peut scraper le revenu brut mondial
    def parse_detail(self, response):
        classement   = response.meta['classement']
        titre        = response.meta['titre']
        rating       = response.meta['rating']
        nb_voteurs   = response.meta['nb_voteurs']

        # Revenu brut mondial – plusieurs variantes pour plus de robustesse
        """
            Explication du sélecteur multiple pour revenu_brut_mondial :
            On utilise une union de plusieurs sélecteurs (séparés par des virgules)
            pour augmenter la robustesse face aux changements de structure ou de langue sur IMDb.

            Partie du sélecteur                                      | Ce qu'elle cible                                              | Quand elle est utile
            ----------------------------------------------------------|---------------------------------------------------------------|------------------------------------------------------
            li[data-testid="title-boxoffice-cumulativeworldwidegross"]| L'élément avec l'attribut data-testid spécifique              | Version la plus fiable quand IMDb utilise cet attribut
            span.ipc-metadata-list-item__list-content-item::text      | Toute span de valeur dans la liste des métadonnées            | Cas où le data-testid est absent ou changé, mais la classe de contenu reste
            li:contains("Gross worldwide")::text                      | N'importe quel <li> contenant exactement ce texte anglais     | Quand le site est en anglais et pas de data-testid
            li:contains("Montant brut mondial")::text                 | N'importe quel <li> contenant ce texte français               | Quand la page est en français
            li:contains("Cumulative Worldwide Gross")::text           | Variante anglaise plus longue (parfois utilisée)              | Anciennes versions ou variantes de la page

            Plus on ajoute de variantes, moins on risque de tout perdre si IMDb change une formulation.
        """
        revenu_brut_mondial = response.css(
            'li[data-testid="title-boxoffice-cumulativeworldwidegross"] '
            'span.ipc-metadata-list-item__list-content-item::text, '
            'li:contains("Gross worldwide")::text, '
            'li:contains("Montant brut mondial")::text, '
            'li:contains("Cumulative Worldwide Gross")::text'
        ).get(default='?').strip()

        # Nettoyage léger
        if revenu_brut_mondial != '?':
            revenu_brut_mondial = revenu_brut_mondial.replace('\xa0', ' ').strip()

        yield {
            'classement': classement,
            'title': titre,
            'url_details': response.url,
            'revenu_brut_mondial': revenu_brut_mondial,
            'rating': rating,
            'nb_voters': nb_voteurs
        }

5. On peut créer un autre spider  `imdb2.py` pour pour scraper d'autres données (ex: les acteurs au lieu des réalisateurs).

Se placer dans le dossier racine du projet (le 1er imdb_scraper/) pour lancer les commandes Scrapy :
```bash
imdb_scraper/                     ← Dossier racine du projet (celui à ouvrir dans VS Code)
├── scrapy.cfg                    
└── imdb_scraper/                 
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py               
    └── spiders/
        ├── __init__.py
        └── spiders (tes spiders ici)
```
ex:
```bash
cd semaine2\scrapy\movie_director_exercice\imdb_scraper
```
```bash
scrapy genspider imdb2 imdb.com
```
la sortie est celle-ci :
```
Created spider 'imdb2' using template 'basic' in module:
  imdb_scraper.spiders.imdb2
```
#### cela a généré le spider (imdb2.py) dans le dossier **spiders**.
-   Modifier / compléter ce fichier qui contient une classe **Imdb1Spider** qui hérite de **scrapy.Spider**
-   Vérifier avec :
```bash
scrapy list
imdb1
imdb2
```
on modifie le fichier imdb2.py pour créer un autre spider qui va scraper d'autres données 
Pour info voici le contenu de ce fichier après la création du spider (avant modification) :
```python
import scrapy


class Imdb2Spider(scrapy.Spider):
    name = "imdb2"
    allowed_domains = ["imdb.com"]
    start_urls = ["https://imdb.com"]

    def parse(self, response):
        pass
